In [80]:
import torch
import numpy as np
import torch.nn as nn 
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.nn import functional as F 
from torch import nn, optim

In [81]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Load Dataset

In [82]:
import torch.utils
import torch.utils.data


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=1024, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, download=True,
                   transform=transforms.ToTensor()),
    batch_size=1024
)

In [83]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 256)
        self.fc22 = nn.Linear(400, 256)
        self.fc3 = nn.Linear(256, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1) # mean, var
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5**logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.relu(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [95]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1+logvar - mu**2 - logvar.exp())
    return BCE + KLD

In [96]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [98]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        # if batch_idx % 110 == 0:
        #     print('Epoch: {}\tBatch_idx: {}\tLoss: {:.4f}'.format(epoch, batch_idx, (loss.item()/len(data))))

    torch.save(model.state_dict(), 'save\\vae.pt')

    print('Epoch: {}, \t Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))


In [112]:
epochs = 100

In [111]:
model.load_state_dict(torch.load('save\\vae.pt'))
for epoch in range(epochs):  
    train(epoch + 1)

Epoch: 1, 	 Average loss: 30.5089
Epoch: 2, 	 Average loss: 30.4987
Epoch: 3, 	 Average loss: 30.3884
Epoch: 4, 	 Average loss: 30.3972
Epoch: 5, 	 Average loss: 30.3802
Epoch: 6, 	 Average loss: 30.3716
Epoch: 7, 	 Average loss: 30.3699
Epoch: 8, 	 Average loss: 30.3384


In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

            if i == 0:
                n = min(data.size(0), 16)
                comparison = torch.cat([data[:n],
                                        recon_batch.view(1024, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                           'results/reconstruction_' + str(epoch) + '.png')
            
    test_loss /= len(test_loader.dataset)
    print('Test set loss: {:4f}'.format(test_loss))

In [108]:
epochs=8
model.load_state_dict(torch.load('save\\vae.pt'))
for epoch in range(epochs):  
    test(epoch + 1)

Test set loss: 30.515961
Test set loss: 30.464157
Test set loss: 30.529575
Test set loss: 30.509229
Test set loss: 30.491249
Test set loss: 30.439775
Test set loss: 30.481024
Test set loss: 30.572414
